In [6]:

# Init
# choropleth
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import seaborn as sns

# County level data retrieval

In [7]:
df_road_2019 = pd.read_csv('./import_data/2019_tiger_road_data.csv', na_values=".")

In [8]:
df = pd.read_csv('./acs_transportation_data/2019-5yr-transport.csv', na_values=".", skiprows=[0])

In [ ]:
df = df.merge(df_road_2019.rename(columns={'fips': 'Geo_FIPS'}), on="Geo_FIPS", how="left")

In [ ]:
# 2019 Data Cleanup

# ACS19_5yr_B01001001 - total_population
df['total_population'] = df['ACS19_5yr_B01001001']
df['log_total_population'] = np.log(df['total_population'])
df['population_density'] = df['ACS19_5yr_B01001001'] / df['Geo_AREALAND']

# Workers 16 Years and Over in Households,ACS19_5yr_B08014001
df['total_workers'] = df['ACS19_5yr_B08014001']

df['land_area'] = df['Geo_AREALAND']# / 1000000 # convert to sqkm

# "Workers Whose Means of Transportation Is Car, Truck, or Van: Aggregate Number of Vehicles (Car, Truck, or Van) Used in Commuting",ACS19_5yr_B08015001
df['aggregate_num_vehicles'] = df['ACS19_5yr_B08015001']

# aggregate_time_per_worker
# Workers 16 Years and Over Who Did Not Work At Home: Aggregate Travel Time to Work (in Minutes).4,ACS19_5yr_B08136001
df['aggregate_time_commute'] = df['ACS19_5yr_B08136001']

# "% Workers 16 Years and Over Who Did Not Work At Home: Aggregate Travel Time to Work (in Minutes): Car, Truck, or Van - Drove Alone",ACS19_5yr_C08136002
df['aggregate_time_commute_drive_alone'] = df['ACS19_5yr_B08136002']

# "% Workers 16 Years and Over Who Did Not Work At Home: Aggregate Travel Time to Work (in Minutes): Car, Truck, or Van - Carpooled",ACS19_5yr_C08136003
df['aggregate_time_commute_carpool'] = df['ACS19_5yr_B08136003']

# % Workers 16 Years and Over Who Did Not Work At Home: Aggregate Travel Time to Work (in Minutes): Public Transportation (Excluding Taxicab),ACS19_5yr_C08136004
df['aggregate_time_commute_public_transit'] = df['ACS19_5yr_B08136004']

# "% Workers 16 Years and Over Who Did Not Work At Home: Aggregate Travel Time to Work (in Minutes): Taxicab, Motorcycle, Bicycle, Walked, or Other Means",ACS19_5yr_C08136005
df['aggregate_time_commute_other_means'] = df['ACS19_5yr_B08136005']

# "Workers 16 Years and Over Who Did Not Work At Home: Car, Truck, or Van - Drove Alone.1",ACS19_5yr_C08134011
df['total_workers_drove_alone'] = df['ACS19_5yr_B08134011']

# "Workers 16 Years and Over Who Did Not Work At Home: Car, Truck, or Van - Carpooled.1",ACS19_5yr_C08134021
df['total_workers_carpooled'] = df['ACS19_5yr_B08134021']

# Workers 16 Years and Over Who Did Not Work At Home: Public Transportation (Excluding Taxicab).1,ACS19_5yr_C08134031
df['total_workers_public_transit'] = df['ACS19_5yr_B08134031']

# "Workers 16 Years and Over Who Did Not Work At Home: Taxicab, Motorcycle, Bicycle,  Walked, or Other Means",ACS19_5yr_C08134041
df['total_workers_other_means'] = df['ACS19_5yr_B08134041']

# Total.181,ACS19_5yr_B08134001
df['ttl_wrkr_cmute'] = df['ACS19_5yr_B08134001']


# Car Truck Or Van - Drove Alone.13,ACS19_5yr_B08122005
df['wrkrs_drove_alone'] = df['ACS19_5yr_B08122005']
# Car Truck Or Van - Drove Alone: Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122006
df['wrkrs_drove_alone_below_100_poverty'] = df['ACS19_5yr_B08122006']
df['pct_wrkrs_drove_alone_below_100_poverty'] = df['ACS19_5yr_B08122006'] / df['ttl_wrkr_cmute']
# Car Truck Or Van - Drove Alone: 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122007
df['wrkrs_drove_alone_100_to_150_poverty'] = df['ACS19_5yr_B08122007']
# Car Truck Or Van - Drove Alone: At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122008
df['wrkrs_drove_alone_over_150_poverty'] = df['ACS19_5yr_B08122008']

# Car Truck Or Van - Carpooled.13,ACS19_5yr_B08122009
df['wrkrs_carpool'] = df['ACS19_5yr_B08122009']
# Car Truck Or Van - Carpooled: Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122010
df['wrkrs_carpool_below_100_poverty'] = df['ACS19_5yr_B08122010']
df['pct_wrkrs_carpool_below_100_poverty'] = df['ACS19_5yr_B08122010'] / df['ttl_wrkr_cmute']
# Car Truck Or Van - Carpooled: 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122011
df['wrkrs_carpool_100_to_150_poverty'] = df['ACS19_5yr_B08122011']
# Car Truck Or Van - Carpooled: At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122012
df['wrkrs_carpool_over_150_poverty'] = df['ACS19_5yr_B08122012']

# Public Transportation (Excluding Taxicab).14,ACS19_5yr_B08122013
df['wrkrs_public_transit'] = df['ACS19_5yr_B08122013']
# Public Transportation (Excluding Taxicab): Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122014
df['wrkrs_public_transit_below_100_poverty'] = df['ACS19_5yr_B08122014']
df['pct_wrkrs_public_transit_below_100_poverty'] = df['ACS19_5yr_B08122014'] / df['ttl_wrkr_cmute']
# Public Transportation (Excluding Taxicab): 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122015
df['wrkrs_public_transit_100_to_150_poverty'] = df['ACS19_5yr_B08122015']
# Public Transportation (Excluding Taxicab): At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122016
df['wrkrs_public_transit_over_150_poverty'] = df['ACS19_5yr_B08122016']

# Walked.14,ACS19_5yr_B08122017
df['wrkrs_walked'] = df['ACS19_5yr_B08122017']
# Walked: Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122018
df['wrkrs_walked_below_100_poverty'] = df['ACS19_5yr_B08122018']
df['pct_wrkrs_walked_below_100_poverty'] = df['ACS19_5yr_B08122018'] / df['ttl_wrkr_cmute']
# Walked: 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122019
df['wrkrs_walked_100_to_150_poverty'] = df['ACS19_5yr_B08122019']
# Walked: At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122020
df['wrkrs_walked_over_150_poverty'] = df['ACS19_5yr_B08122020']

# Taxicab Motorcycle Bicycle Or Other Means.13,ACS19_5yr_B08122021
df['wrkrs_commute_other_means'] = df['ACS19_5yr_B08122021']
# Taxicab Motorcycle Bicycle Or Other Means: Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122022
df['wrkrs_commute_other_means_below_100_poverty'] = df['ACS19_5yr_B08122022']
df['pct_wrkrs_commute_other_means_below_100_poverty'] = df['ACS19_5yr_B08122022'] / df['ttl_wrkr_cmute']
# Taxicab Motorcycle Bicycle Or Other Means: 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122023
df['wrkrs_commute_other_means_100_to_150_poverty'] = df['ACS19_5yr_B08122023']
# Taxicab Motorcycle Bicycle Or Other Means: At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122024
df['wrkrs_commute_other_means_over_150_poverty'] = df['ACS19_5yr_B08122024']

# # Worked From Home.14,ACS19_5yr_B08122025
# df['total_workers_other_means'] = df['ACS19_5yr_B08134041']
# # Worked From Home Below 100 Percent Of The Poverty Level,ACS19_5yr_B08122026
# df['total_workers_other_means'] = df['ACS19_5yr_B08134041']
# # Worked From Home 100 To 149 Percent Of The Poverty Level,ACS19_5yr_B08122027
# df['total_workers_other_means'] = df['ACS19_5yr_B08134041']
# # Worked From Home At Or Above 150 Percent Of The Poverty Level,ACS19_5yr_B08122028
# df['total_workers_other_means'] = df['ACS19_5yr_B08134041']

# Less Than 10 Minutes,ACS19_5yr_B08134002
df['ttl_wrkr_cmute_under_10_min'] = df['ACS19_5yr_B08134002']
# 10 To 14 Minutes.1,ACS19_5yr_B08134003
df['ttl_wrkr_cmute_10_to_14_min'] = df['ACS19_5yr_B08134003']
# 15 To 19 Minutes.1,ACS19_5yr_B08134004
df['ttl_wrkr_cmute_15_to_14_min'] = df['ACS19_5yr_B08134004']
# 20 To 24 Minutes.1,ACS19_5yr_B08134005
df['ttl_wrkr_cmute_20_to_24_min'] = df['ACS19_5yr_B08134005']
# 25 To 29 Minutes.1,ACS19_5yr_B08134006
df['ttl_wrkr_cmute_25_to_29_min'] = df['ACS19_5yr_B08134006']
# 30 To 34 Minutes.1,ACS19_5yr_B08134007
df['ttl_wrkr_cmute_30_to_34_min'] = df['ACS19_5yr_B08134007']
# 35 To 44 Minutes,ACS19_5yr_B08134008
df['ttl_wrkr_cmute_35_to_44_min'] = df['ACS19_5yr_B08134008']
# 45 To 59 Minutes.1,ACS19_5yr_B08134009
df['ttl_wrkr_cmute_45_to_59_min'] = df['ACS19_5yr_B08134009']
# 60 Or More Minutes,ACS19_5yr_B08134010
df['ttl_wrkr_cmute_over_60'] = df['ACS19_5yr_B08134010']
# 15 min range Percents
df['pct_wrkr_cmute_01_to_14_min'] = (df['ttl_wrkr_cmute_under_10_min'] + df['ttl_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_wrkr_cmute_15_to_29_min'] = (df['ttl_wrkr_cmute_15_to_14_min'] + df['ttl_wrkr_cmute_20_to_24_min'] + df['ttl_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_wrkr_cmute_30_to_44_min'] = (df['ttl_wrkr_cmute_30_to_34_min'] + df['ttl_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_wrkr_cmute_44_to_59_min'] = df['ttl_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_wrkr_cmute_over_60_min'] = df['ttl_wrkr_cmute_over_60'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_wrkr_cmute_01_to_14_min'] = (df['ttl_wrkr_cmute_under_10_min'] + df['ttl_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
# df['pct_wrkr_cmute_15_to_29_min'] = (df['ttl_wrkr_cmute_15_to_14_min'] + df['ttl_wrkr_cmute_20_to_24_min'] + df['ttl_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
# df['pct_wrkr_cmute_30_to_44_min'] = (df['ttl_wrkr_cmute_30_to_34_min'] + df['ttl_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
# df['pct_wrkr_cmute_44_to_59_min'] = df['ttl_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
# df['pct_wrkr_cmute_over_60_min'] = df['ttl_wrkr_cmute_over_60'] / df['ttl_wrkr_cmute']

# Car Truck Or Van.1,ACS19_5yr_B08134011
df['ttl_car_wrkr_cmute'] = df['ACS19_5yr_B08134011']
# Car Truck Or Van: Less Than 10 Minutes,ACS19_5yr_B08134012
df['ttl_car_wrkr_cmute_under_10_min'] = df['ACS19_5yr_B08134012']
# Car Truck Or Van: 10 To 14 Minutes,ACS19_5yr_B08134013
df['ttl_car_wrkr_cmute_10_to_14_min'] = df['ACS19_5yr_B08134013']
# Car Truck Or Van: 15 To 19 Minutes,ACS19_5yr_B08134014
df['ttl_car_wrkr_cmute_15_to_14_min'] = df['ACS19_5yr_B08134014']
# Car Truck Or Van: 20 To 24 Minutes,ACS19_5yr_B08134015
df['ttl_car_wrkr_cmute_20_to_24_min'] = df['ACS19_5yr_B08134015']
# Car Truck Or Van: 25 To 29 Minutes,ACS19_5yr_B08134016
df['ttl_car_wrkr_cmute_25_to_29_min'] = df['ACS19_5yr_B08134016']
# Car Truck Or Van: 30 To 34 Minutes,ACS19_5yr_B08134017
df['ttl_car_wrkr_cmute_30_to_34_min'] = df['ACS19_5yr_B08134017']
# Car Truck Or Van: 35 To 44 Minutes,ACS19_5yr_B08134018
df['ttl_car_wrkr_cmute_35_to_44_min'] = df['ACS19_5yr_B08134018']
# Car Truck Or Van: 45 To 59 Minutes,ACS19_5yr_B08134019
df['ttl_car_wrkr_cmute_45_to_59_min'] = df['ACS19_5yr_B08134019']
# Car Truck Or Van: 60 Or More Minutes,ACS19_5yr_B08134020
df['ttl_car_wrkr_cmute_over_60_min'] = df['ACS19_5yr_B08134020']
# # Car Truck Or Van: Drove Alone.1,ACS19_5yr_B08134021
# df['ttl_car_wrkr_cmute'] = df['ACS19_5yr_B08134021']
# # Car Truck Or Van: Drove Alone: Less Than 10 Minutes,ACS19_5yr_B08134022
# df['ttl_car_wrkr_cmute_under_10_min'] = df['ACS19_5yr_B08134022']
# # Car Truck Or Van: Drove Alone: 10 To 14 Minutes,ACS19_5yr_B08134023
# df['ttl_car_wrkr_cmute_10_to_14_min'] = df['ACS19_5yr_B08134023']
# # Car Truck Or Van: Drove Alone: 15 To 19 Minutes,ACS19_5yr_B08134024
# df['ttl_car_wrkr_cmute_15_to_14_min'] = df['ACS19_5yr_B08134024']
# # Car Truck Or Van: Drove Alone: 20 To 24 Minutes,ACS19_5yr_B08134025
# df['ttl_car_wrkr_cmute_20_to_24_min'] = df['ACS19_5yr_B08134025']
# # Car Truck Or Van: Drove Alone: 25 To 29 Minutes,ACS19_5yr_B08134026
# df['ttl_car_wrkr_cmute_25_to_29_min'] = df['ACS19_5yr_B08134026']
# # Car Truck Or Van: Drove Alone: 30 To 34 Minutes,ACS19_5yr_B08134027
# df['ttl_car_wrkr_cmute_30_to_34_min'] = df['ACS19_5yr_B08134027']
# # Car Truck Or Van: Drove Alone: 35 To 44 Minutes,ACS19_5yr_B08134028
# df['ttl_car_wrkr_cmute_35_to_44_min'] = df['ACS19_5yr_B08134028']
# # Car Truck Or Van: Drove Alone: 45 To 59 Minutes,ACS19_5yr_B08134029
# df['ttl_car_wrkr_cmute_45_to_59_min'] = df['ACS19_5yr_B08134029']
# # Car Truck Or Van: Drove Alone: 60 Or More Minutes,ACS19_5yr_B08134030
# df['ttl_car_wrkr_cmute_over_60_min'] = df['ACS19_5yr_B08134030']
# 15 min range Percents
df['pct_car_wrkr_cmute'] = df['ttl_car_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_car_wrkr_cmute_01_to_14_min'] = (df['ttl_car_wrkr_cmute_under_10_min'] + df['ttl_car_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_car_wrkr_cmute_15_to_29_min'] = (df['ttl_car_wrkr_cmute_15_to_14_min'] + df['ttl_car_wrkr_cmute_20_to_24_min'] + df['ttl_car_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_car_wrkr_cmute_30_to_44_min'] = (df['ttl_car_wrkr_cmute_30_to_34_min'] + df['ttl_car_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_car_wrkr_cmute_44_to_59_min'] = df['ttl_car_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_car_wrkr_cmute_over_60_min'] = df['ttl_car_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']

df['log_pct_car_wrkr_cmute']               = np.log(df['pct_car_wrkr_cmute'])
df['log_pct_car_wrkr_cmute_01_to_14_min']  = np.log(df['pct_car_wrkr_cmute_01_to_14_min'])
df['log_pct_car_wrkr_cmute_15_to_29_min']  = np.log(df['pct_car_wrkr_cmute_15_to_29_min'])
df['log_pct_car_wrkr_cmute_30_to_44_min']  = np.log(df['pct_car_wrkr_cmute_30_to_44_min'])
df['log_pct_car_wrkr_cmute_44_to_59_min']  = np.log(df['pct_car_wrkr_cmute_44_to_59_min'])
df['log_pct_car_wrkr_cmute_over_60_min']   = np.log(df['pct_car_wrkr_cmute_over_60_min'])
# # 15 min range Percents
# df['pct_car_wrkr_cmute_01_to_14_min'] = (df['ttl_car_wrkr_cmute_under_10_min'] + df['ttl_car_wrkr_cmute_10_to_14_min']) / df['ttl_car_wrkr_cmute']
# df['pct_car_wrkr_cmute_15_to_29_min'] = (df['ttl_car_wrkr_cmute_15_to_14_min'] + df['ttl_car_wrkr_cmute_20_to_24_min'] + df['ttl_car_wrkr_cmute_25_to_29_min']) / df['ttl_car_wrkr_cmute']
# df['pct_car_wrkr_cmute_30_to_44_min'] = (df['ttl_car_wrkr_cmute_30_to_34_min'] + df['ttl_car_wrkr_cmute_35_to_44_min']) / df['ttl_car_wrkr_cmute']
# df['pct_car_wrkr_cmute_44_to_59_min'] = df['ttl_car_wrkr_cmute_45_to_59_min'] / df['ttl_car_wrkr_cmute']
# df['pct_car_wrkr_cmute_over_60_min'] = df['ttl_car_wrkr_cmute_over_60_min'] / df['ttl_car_wrkr_cmute']

df['zero_check'] = 1
# Zero Check
df['zero_check'] = df['pct_car_wrkr_cmute_01_to_14_min'] * \
                        df['pct_car_wrkr_cmute_15_to_29_min'] * \
                        df['pct_car_wrkr_cmute_30_to_44_min'] * \
                        df['pct_car_wrkr_cmute_44_to_59_min'] * \
                        df['pct_car_wrkr_cmute_over_60_min'] * df['zero_check']

##### SOLO ABD CARPOOL DRIVIGN IS COMBINED INTO OVERALL CAR

# #  Carpooled,ACS19_5yr_B08134031
# df['ttl_carpool_wrkr_cmute'] = df['ACS19_5yr_B08134031']
# #  Carpooled: Less Than 10 Minutes,ACS19_5yr_B08134032
# df['ttl_carpool_wrkr_cmute_under_10_min'] = df['ACS19_5yr_B08134032']
# #  Carpooled: 10 To 14 Minutes,ACS19_5yr_B08134033
# df['ttl_carpool_wrkr_cmute_10_to_14_min'] = df['ACS19_5yr_B08134033']
# #  Carpooled: 15 To 19 Minutes,ACS19_5yr_B08134034
# df['ttl_carpool_wrkr_cmute_15_to_14_min'] = df['ACS19_5yr_B08134034']
# #  Carpooled: 20 To 24 Minutes,ACS19_5yr_B08134035
# df['ttl_carpool_wrkr_cmute_20_to_24_min'] = df['ACS19_5yr_B08134035']
# #  Carpooled: 25 To 29 Minutes,ACS19_5yr_B08134036
# df['ttl_carpool_wrkr_cmute_25_to_29_min'] = df['ACS19_5yr_B08134036']
# #  Carpooled: 30 To 34 Minutes,ACS19_5yr_B08134037
# df['ttl_carpool_wrkr_cmute_30_to_34_min'] = df['ACS19_5yr_B08134037']
# #  Carpooled: 35 To 44 Minutes,ACS19_5yr_B08134038
# df['ttl_carpool_wrkr_cmute_35_to_44_min'] = df['ACS19_5yr_B08134038']
# #  Carpooled: 45 To 59 Minutes,ACS19_5yr_B08134039
# df['ttl_carpool_wrkr_cmute_45_to_59_min'] = df['ACS19_5yr_B08134039']
# #  Carpooled: 60 Or More Minutes,ACS19_5yr_B08134040
# df['ttl_carpool_wrkr_cmute_over_60_min'] = df['ACS19_5yr_B08134040']
# # 15 min range Percents
# df['pct_carpool_wrkr_cmute'] = df['ttl_carpool_wrkr_cmute'] / df['ttl_wrkr_cmute']
# df['pct_carpool_wrkr_cmute_01_to_14_min'] = (df['ttl_carpool_wrkr_cmute_under_10_min'] + df['ttl_carpool_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
# df['pct_carpool_wrkr_cmute_15_to_29_min'] = (df['ttl_carpool_wrkr_cmute_15_to_14_min'] + df['ttl_carpool_wrkr_cmute_20_to_24_min'] + df['ttl_carpool_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
# df['pct_carpool_wrkr_cmute_30_to_44_min'] = (df['ttl_carpool_wrkr_cmute_30_to_34_min'] + df['ttl_carpool_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
# df['pct_carpool_wrkr_cmute_44_to_59_min'] = df['ttl_carpool_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
# df['pct_carpool_wrkr_cmute_over_60_min'] = df['ttl_carpool_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # # 15 min range Percents
# # df['pct_carpool_wrkr_cmute_01_to_14_min'] = (df['ttl_carpool_wrkr_cmute_under_10_min'] + df['ttl_carpool_wrkr_cmute_10_to_14_min']) / df['ttl_carpool_wrkr_cmute']
# # df['pct_carpool_wrkr_cmute_15_to_29_min'] = (df['ttl_carpool_wrkr_cmute_15_to_14_min'] + df['ttl_carpool_wrkr_cmute_20_to_24_min'] + df['ttl_carpool_wrkr_cmute_25_to_29_min']) / df['ttl_carpool_wrkr_cmute']
# # df['pct_carpool_wrkr_cmute_30_to_44_min'] = (df['ttl_carpool_wrkr_cmute_30_to_34_min'] + df['ttl_carpool_wrkr_cmute_35_to_44_min']) / df['ttl_carpool_wrkr_cmute']
# # df['pct_carpool_wrkr_cmute_44_to_59_min'] = df['ttl_carpool_wrkr_cmute_45_to_59_min'] / df['ttl_carpool_wrkr_cmute']
# # df['pct_carpool_wrkr_cmute_over_60_min'] = df['ttl_carpool_wrkr_cmute_over_60_min'] / df['ttl_carpool_wrkr_cmute']


# Public Transportation (Excluding Taxicab).20,ACS19_5yr_B08134061
df['ttl_all_public_tspt_wrkr_cmute'] = df['ACS19_5yr_B08134061']
# Public Transportation (Excluding Taxicab): Less Than 10 Minutes,ACS19_5yr_B08134062
df['ttl_all_public_tspt_wrkr_cmute_under_10_min'] = df['ACS19_5yr_B08134062']
# Public Transportation (Excluding Taxicab): 10 To 14 Minutes,ACS19_5yr_B08134063
df['ttl_all_public_tspt_wrkr_cmute_10_to_14_min'] = df['ACS19_5yr_B08134063']
# Public Transportation (Excluding Taxicab): 15 To 19 Minutes,ACS19_5yr_B08134064
df['ttl_all_public_tspt_wrkr_cmute_15_to_14_min'] = df['ACS19_5yr_B08134064']
# Public Transportation (Excluding Taxicab): 20 To 24 Minutes,ACS19_5yr_B08134065
df['ttl_all_public_tspt_wrkr_cmute_20_to_24_min'] = df['ACS19_5yr_B08134065']
# Public Transportation (Excluding Taxicab): 25 To 29 Minutes,ACS19_5yr_B08134066
df['ttl_all_public_tspt_wrkr_cmute_25_to_29_min'] = df['ACS19_5yr_B08134066']
# Public Transportation (Excluding Taxicab): 30 To 34 Minutes,ACS19_5yr_B08134067
df['ttl_all_public_tspt_wrkr_cmute_30_to_34_min'] = df['ACS19_5yr_B08134067']
# Public Transportation (Excluding Taxicab): 35 To 44 Minutes,ACS19_5yr_B08134068
df['ttl_all_public_tspt_wrkr_cmute_35_to_44_min'] = df['ACS19_5yr_B08134068']
# Public Transportation (Excluding Taxicab): 45 To 59 Minutes,ACS19_5yr_B08134069
df['ttl_all_public_tspt_wrkr_cmute_45_to_59_min'] = df['ACS19_5yr_B08134069']
# Public Transportation (Excluding Taxicab): 60 Or More Minutes,ACS19_5yr_B08134070
df['ttl_all_public_tspt_wrkr_cmute_over_60_min'] = df['ACS19_5yr_B08134070']
# 15 min range Percents
df['pct_all_public_tspt_wrkr_cmute'] = df['ttl_all_public_tspt_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_all_public_tspt_wrkr_cmute_01_to_14_min'] = (df['ttl_all_public_tspt_wrkr_cmute_under_10_min'] + df['ttl_all_public_tspt_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_all_public_tspt_wrkr_cmute_15_to_29_min'] = (df['ttl_all_public_tspt_wrkr_cmute_15_to_14_min'] + df['ttl_all_public_tspt_wrkr_cmute_20_to_24_min'] + df['ttl_all_public_tspt_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_all_public_tspt_wrkr_cmute_30_to_44_min'] = (df['ttl_all_public_tspt_wrkr_cmute_30_to_34_min'] + df['ttl_all_public_tspt_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_all_public_tspt_wrkr_cmute_44_to_59_min'] = df['ttl_all_public_tspt_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_all_public_tspt_wrkr_cmute_over_60_min'] = df['ttl_all_public_tspt_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_all_public_tspt_wrkr_cmute_01_to_14_min'] = (df['ttl_all_public_tspt_wrkr_cmute_under_10_min'] + df['ttl_all_public_tspt_wrkr_cmute_10_to_14_min']) / df['ttl_all_public_tspt_wrkr_cmute']
# df['pct_all_public_tspt_wrkr_cmute_15_to_29_min'] = (df['ttl_all_public_tspt_wrkr_cmute_15_to_14_min'] + df['ttl_all_public_tspt_wrkr_cmute_20_to_24_min'] + df['ttl_all_public_tspt_wrkr_cmute_25_to_29_min']) / df['ttl_all_public_tspt_wrkr_cmute']
# df['pct_all_public_tspt_wrkr_cmute_30_to_44_min'] = (df['ttl_all_public_tspt_wrkr_cmute_30_to_34_min'] + df['ttl_all_public_tspt_wrkr_cmute_35_to_44_min']) / df['ttl_all_public_tspt_wrkr_cmute']
# df['pct_all_public_tspt_wrkr_cmute_44_to_59_min'] = df['ttl_all_public_tspt_wrkr_cmute_45_to_59_min'] / df['ttl_all_public_tspt_wrkr_cmute']
# df['pct_all_public_tspt_wrkr_cmute_over_60_min'] = df['ttl_all_public_tspt_wrkr_cmute_over_60_min'] / df['ttl_all_public_tspt_wrkr_cmute']
# Zero Check
df['zero_check'] = df['pct_all_public_tspt_wrkr_cmute_01_to_14_min'] * \
                        df['pct_all_public_tspt_wrkr_cmute_15_to_29_min'] * \
                        df['pct_all_public_tspt_wrkr_cmute_30_to_44_min'] * \
                        df['pct_all_public_tspt_wrkr_cmute_44_to_59_min'] * \
                        df['pct_all_public_tspt_wrkr_cmute_over_60_min'] * df['zero_check']


# Public Transportation (Excluding Taxicab): Bus.1,ACS19_5yr_B08134071
df['ttl_bus_wrkr_cmute'] = df['ACS19_5yr_B08134071']
# Public Transportation (Excluding Taxicab): Bus: Less Than 10 Minutes,ACS19_5yr_B08134072
df['ttl_bus_wrkr_cmute_under_10_min'] = df['ACS19_5yr_B08134072']
# Public Transportation (Excluding Taxicab): Bus: 10 To 14 Minutes,ACS19_5yr_B08134073
df['ttl_bus_wrkr_cmute_10_to_14_min'] = df['ACS19_5yr_B08134073']
# Public Transportation (Excluding Taxicab): Bus: 15 To 19 Minutes,ACS19_5yr_B08134074
df['ttl_bus_wrkr_cmute_15_to_14_min'] = df['ACS19_5yr_B08134074']
# Public Transportation (Excluding Taxicab): Bus: 20 To 24 Minutes,ACS19_5yr_B08134075
df['ttl_bus_wrkr_cmute_20_to_24_min'] = df['ACS19_5yr_B08134075']
# Public Transportation (Excluding Taxicab): Bus: 25 To 29 Minutes,ACS19_5yr_B08134076
df['ttl_bus_wrkr_cmute_25_to_29_min'] = df['ACS19_5yr_B08134076']
# Public Transportation (Excluding Taxicab): Bus: 30 To 34 Minutes,ACS19_5yr_B08134077
df['ttl_bus_wrkr_cmute_30_to_34_min'] = df['ACS19_5yr_B08134077']
# Public Transportation (Excluding Taxicab): Bus: 35 To 44 Minutes,ACS19_5yr_B08134078
df['ttl_bus_wrkr_cmute_35_to_44_min'] = df['ACS19_5yr_B08134078']
# Public Transportation (Excluding Taxicab): Bus: 45 To 59 Minutes,ACS19_5yr_B08134079
df['ttl_bus_wrkr_cmute_45_to_59_min'] = df['ACS19_5yr_B08134079']
# Public Transportation (Excluding Taxicab): Bus: 60 Or More Minutes,ACS19_5yr_B08134080
df['ttl_bus_wrkr_cmute_over_60_min'] = df['ACS19_5yr_B08134080']
# 15 min range Percents
df['pct_bus_wrkr_cmute'] = df['ttl_bus_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_bus_wrkr_cmute_01_to_14_min'] = (df['ttl_bus_wrkr_cmute_under_10_min'] + df['ttl_bus_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_bus_wrkr_cmute_15_to_29_min'] = (df['ttl_bus_wrkr_cmute_15_to_14_min'] + df['ttl_bus_wrkr_cmute_20_to_24_min'] + df['ttl_bus_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_bus_wrkr_cmute_30_to_44_min'] = (df['ttl_bus_wrkr_cmute_30_to_34_min'] + df['ttl_bus_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_bus_wrkr_cmute_44_to_59_min'] = df['ttl_bus_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_bus_wrkr_cmute_over_60_min'] = df['ttl_bus_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_bus_wrkr_cmute_01_to_14_min'] = (df['ttl_bus_wrkr_cmute_under_10_min'] + df['ttl_bus_wrkr_cmute_10_to_14_min']) / df['ttl_bus_wrkr_cmute']
# df['pct_bus_wrkr_cmute_15_to_29_min'] = (df['ttl_bus_wrkr_cmute_15_to_14_min'] + df['ttl_bus_wrkr_cmute_20_to_24_min'] + df['ttl_bus_wrkr_cmute_25_to_29_min']) / df['ttl_bus_wrkr_cmute']
# df['pct_bus_wrkr_cmute_30_to_44_min'] = (df['ttl_bus_wrkr_cmute_30_to_34_min'] + df['ttl_bus_wrkr_cmute_35_to_44_min']) / df['ttl_bus_wrkr_cmute']
# df['pct_bus_wrkr_cmute_44_to_59_min'] = df['ttl_bus_wrkr_cmute_45_to_59_min'] / df['ttl_bus_wrkr_cmute']
# df['pct_bus_wrkr_cmute_over_60_min'] = df['ttl_bus_wrkr_cmute_over_60_min'] / df['ttl_bus_wrkr_cmute']
# Zero Check
df['zero_check'] = df['pct_bus_wrkr_cmute_01_to_14_min'] * \
                        df['pct_bus_wrkr_cmute_15_to_29_min'] * \
                        df['pct_bus_wrkr_cmute_30_to_44_min'] * \
                        df['pct_bus_wrkr_cmute_44_to_59_min'] * \
                        df['pct_bus_wrkr_cmute_over_60_min'] * df['zero_check']


#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico),ACS19_5yr_B08134081
df['ttl_local_rails_wrkr_cmute'] = df['ACS19_5yr_B08134081']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): Less Than 10 Minutes,ACS19_5yr_B08134082
df['ttl_local_rails_wrkr_cmute_under_10_min'] = df['ACS19_5yr_B08134082']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 10 To 14 Minutes,ACS19_5yr_B08134083
df['ttl_local_rails_wrkr_cmute_10_to_14_min'] = df['ACS19_5yr_B08134083']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 15 To 19 Minutes,ACS19_5yr_B08134084
df['ttl_local_rails_wrkr_cmute_15_to_14_min'] = df['ACS19_5yr_B08134084']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 20 To 24 Minutes,ACS19_5yr_B08134085
df['ttl_local_rails_wrkr_cmute_20_to_24_min'] = df['ACS19_5yr_B08134085']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 25 To 29 Minutes,ACS19_5yr_B08134086
df['ttl_local_rails_wrkr_cmute_25_to_29_min'] = df['ACS19_5yr_B08134086']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 30 To 34 Minutes,ACS19_5yr_B08134087
df['ttl_local_rails_wrkr_cmute_30_to_34_min'] = df['ACS19_5yr_B08134087']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 35 To 44 Minutes,ACS19_5yr_B08134088
df['ttl_local_rails_wrkr_cmute_35_to_44_min'] = df['ACS19_5yr_B08134088']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 45 To 59 Minutes,ACS19_5yr_B08134089
df['ttl_local_rails_wrkr_cmute_45_to_59_min'] = df['ACS19_5yr_B08134089']
#  Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro P̼blico In Puerto Rico): 60 Or More Minutes,ACS19_5yr_B08134090
df['ttl_local_rails_wrkr_cmute_over_60_min'] = df['ACS19_5yr_B08134090']
# 15 min range Percents
df['pct_local_rails_wrkr_cmute'] = df['ttl_local_rails_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_local_rails_wrkr_cmute_01_to_14_min'] = (df['ttl_local_rails_wrkr_cmute_under_10_min'] + df['ttl_local_rails_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_local_rails_wrkr_cmute_15_to_29_min'] = (df['ttl_local_rails_wrkr_cmute_15_to_14_min'] + df['ttl_local_rails_wrkr_cmute_20_to_24_min'] + df['ttl_local_rails_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_local_rails_wrkr_cmute_30_to_44_min'] = (df['ttl_local_rails_wrkr_cmute_30_to_34_min'] + df['ttl_local_rails_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_local_rails_wrkr_cmute_44_to_59_min'] = df['ttl_local_rails_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_local_rails_wrkr_cmute_over_60_min'] = df['ttl_local_rails_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_local_rails_wrkr_cmute_01_to_14_min'] = (df['ttl_local_rails_wrkr_cmute_under_10_min'] + df['ttl_local_rails_wrkr_cmute_10_to_14_min']) / df['ttl_local_rails_wrkr_cmute']
# df['pct_local_rails_wrkr_cmute_15_to_29_min'] = (df['ttl_local_rails_wrkr_cmute_15_to_14_min'] + df['ttl_local_rails_wrkr_cmute_20_to_24_min'] + df['ttl_local_rails_wrkr_cmute_25_to_29_min']) / df['ttl_local_rails_wrkr_cmute']
# df['pct_local_rails_wrkr_cmute_30_to_44_min'] = (df['ttl_local_rails_wrkr_cmute_30_to_34_min'] + df['ttl_local_rails_wrkr_cmute_35_to_44_min']) / df['ttl_local_rails_wrkr_cmute']
# df['pct_local_rails_wrkr_cmute_44_to_59_min'] = df['ttl_local_rails_wrkr_cmute_45_to_59_min'] / df['ttl_local_rails_wrkr_cmute']
# df['pct_local_rails_wrkr_cmute_over_60_min'] = df['ttl_local_rails_wrkr_cmute_over_60_min'] / df['ttl_local_rails_wrkr_cmute']
# Zero Check
df['zero_check'] = df['pct_local_rails_wrkr_cmute_01_to_14_min'] * \
                        df['pct_local_rails_wrkr_cmute_15_to_29_min'] * \
                        df['pct_local_rails_wrkr_cmute_30_to_44_min'] * \
                        df['pct_local_rails_wrkr_cmute_44_to_59_min'] * \
                        df['pct_local_rails_wrkr_cmute_over_60_min'] * df['zero_check']


#  Long-Distance Train Or Commuter Rail Or Ferryboat,ACS19_5yr_B08134091
df['ttl_long_distance_public_transport_wrkr_cmute'] = df['ACS19_5yr_B08134091']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: Less Than 10 Minutes,ACS19_5yr_B08134092
df['ttl_long_distance_public_transport_wrkr_cmute_under_10_min'] = df['ACS19_5yr_B08134092']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 10 To 14 Minutes,ACS19_5yr_B08134093
df['ttl_long_distance_public_transport_wrkr_cmute_10_to_14_min'] = df['ACS19_5yr_B08134093']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 15 To 19 Minutes,ACS19_5yr_B08134094
df['ttl_long_distance_public_transport_wrkr_cmute_15_to_14_min'] = df['ACS19_5yr_B08134094']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 20 To 24 Minutes,ACS19_5yr_B08134095
df['ttl_long_distance_public_transport_wrkr_cmute_20_to_24_min'] = df['ACS19_5yr_B08134095']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 25 To 29 Minutes,ACS19_5yr_B08134096
df['ttl_long_distance_public_transport_wrkr_cmute_25_to_29_min'] = df['ACS19_5yr_B08134096']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 30 To 34 Minutes,ACS19_5yr_B08134097
df['ttl_long_distance_public_transport_wrkr_cmute_30_to_34_min'] = df['ACS19_5yr_B08134097']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 35 To 44 Minutes,ACS19_5yr_B08134098
df['ttl_long_distance_public_transport_wrkr_cmute_35_to_44_min'] = df['ACS19_5yr_B08134098']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 45 To 59 Minutes,ACS19_5yr_B08134099
df['ttl_long_distance_public_transport_wrkr_cmute_45_to_59_min'] = df['ACS19_5yr_B08134099']
#  Long-Distance Train Or Commuter Rail Or Ferryboat: 60 Or More Minutes,ACS19_5yr_B08134100
df['ttl_long_distance_public_transport_wrkr_cmute_over_60_min'] = df['ACS19_5yr_B08134100']
# 15 min range Percents
df['pct_long_distance_public_transport_wrkr_cmute'] = df['ttl_long_distance_public_transport_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_long_distance_public_transport_wrkr_cmute_01_to_14_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_under_10_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_long_distance_public_transport_wrkr_cmute_15_to_29_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_15_to_14_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_20_to_24_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_long_distance_public_transport_wrkr_cmute_30_to_44_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_30_to_34_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_long_distance_public_transport_wrkr_cmute_44_to_59_min'] = df['ttl_long_distance_public_transport_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_long_distance_public_transport_wrkr_cmute_over_60_min'] = df['ttl_long_distance_public_transport_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_long_distance_public_transport_wrkr_cmute_01_to_14_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_under_10_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_10_to_14_min']) / df['ttl_long_distance_public_transport_wrkr_cmute']
# df['pct_long_distance_public_transport_wrkr_cmute_15_to_29_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_15_to_14_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_20_to_24_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_25_to_29_min']) / df['ttl_long_distance_public_transport_wrkr_cmute']
# df['pct_long_distance_public_transport_wrkr_cmute_30_to_44_min'] = (df['ttl_long_distance_public_transport_wrkr_cmute_30_to_34_min'] + df['ttl_long_distance_public_transport_wrkr_cmute_35_to_44_min']) / df['ttl_long_distance_public_transport_wrkr_cmute']
# df['pct_long_distance_public_transport_wrkr_cmute_44_to_59_min'] = df['ttl_long_distance_public_transport_wrkr_cmute_45_to_59_min'] / df['ttl_long_distance_public_transport_wrkr_cmute']
# df['pct_long_distance_public_transport_wrkr_cmute_over_60_min'] = df['ttl_long_distance_public_transport_wrkr_cmute_over_60_min'] / df['ttl_long_distance_public_transport_wrkr_cmute']
# Zero Check
df['zero_check'] = df['pct_long_distance_public_transport_wrkr_cmute_01_to_14_min'] * \
                        df['pct_long_distance_public_transport_wrkr_cmute_15_to_29_min'] * \
                        df['pct_long_distance_public_transport_wrkr_cmute_30_to_44_min'] * \
                        df['pct_long_distance_public_transport_wrkr_cmute_44_to_59_min'] * \
                        df['pct_long_distance_public_transport_wrkr_cmute_over_60_min'] * df['zero_check']


# Walked.20,ACS19_5yr_B08134101
df['ttl_walked_wrkr_cmute'] = df['ACS19_5yr_B08134101']
# Walked: Less Than 10 Minutes,ACS19_5yr_B08134102
df['ttl_walked_wrkr_cmute_under_10_min'] = df['ACS19_5yr_B08134102']
# Walked: 10 To 14 Minutes,ACS19_5yr_B08134103
df['ttl_walked_wrkr_cmute_10_to_14_min'] = df['ACS19_5yr_B08134103']
# Walked: 15 To 19 Minutes,ACS19_5yr_B08134104
df['ttl_walked_wrkr_cmute_15_to_14_min'] = df['ACS19_5yr_B08134104']
# Walked: 20 To 24 Minutes,ACS19_5yr_B08134105
df['ttl_walked_wrkr_cmute_20_to_24_min'] = df['ACS19_5yr_B08134105']
# Walked: 25 To 29 Minutes,ACS19_5yr_B08134106
df['ttl_walked_wrkr_cmute_25_to_29_min'] = df['ACS19_5yr_B08134106']
# Walked: 30 To 34 Minutes,ACS19_5yr_B08134107
df['ttl_walked_wrkr_cmute_30_to_34_min'] = df['ACS19_5yr_B08134107']
# Walked: 35 To 44 Minutes,ACS19_5yr_B08134108
df['ttl_walked_wrkr_cmute_35_to_44_min'] = df['ACS19_5yr_B08134108']
# Walked: 45 To 59 Minutes,ACS19_5yr_B08134109
df['ttl_walked_wrkr_cmute_45_to_59_min'] = df['ACS19_5yr_B08134109']
# Walked: 60 Or More Minutes,ACS19_5yr_B08134110
df['ttl_walked_wrkr_cmute_over_60_min'] = df['ACS19_5yr_B08134110']
# 15 min range Percents
df['pct_walked_wrkr_cmute'] = df['ttl_walked_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_walked_wrkr_cmute_01_to_14_min'] = (df['ttl_walked_wrkr_cmute_under_10_min'] + df['ttl_walked_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_walked_wrkr_cmute_15_to_29_min'] = (df['ttl_walked_wrkr_cmute_15_to_14_min'] + df['ttl_walked_wrkr_cmute_20_to_24_min'] + df['ttl_walked_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_walked_wrkr_cmute_30_to_44_min'] = (df['ttl_walked_wrkr_cmute_30_to_34_min'] + df['ttl_walked_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_walked_wrkr_cmute_44_to_59_min'] = df['ttl_walked_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_walked_wrkr_cmute_over_60_min'] = df['ttl_walked_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_walked_wrkr_cmute_01_to_14_min'] = (df['ttl_walked_wrkr_cmute_under_10_min'] + df['ttl_walked_wrkr_cmute_10_to_14_min']) / df['ttl_walked_wrkr_cmute']
# df['pct_walked_wrkr_cmute_15_to_29_min'] = (df['ttl_walked_wrkr_cmute_15_to_14_min'] + df['ttl_walked_wrkr_cmute_20_to_24_min'] + df['ttl_walked_wrkr_cmute_25_to_29_min']) / df['ttl_walked_wrkr_cmute']
# df['pct_walked_wrkr_cmute_30_to_44_min'] = (df['ttl_walked_wrkr_cmute_30_to_34_min'] + df['ttl_walked_wrkr_cmute_35_to_44_min']) / df['ttl_walked_wrkr_cmute']
# df['pct_walked_wrkr_cmute_44_to_59_min'] = df['ttl_walked_wrkr_cmute_45_to_59_min'] / df['ttl_walked_wrkr_cmute']
# df['pct_walked_wrkr_cmute_over_60_min'] = df['ttl_walked_wrkr_cmute_over_60_min'] / df['ttl_walked_wrkr_cmute']
# Zero Check
df['zero_check'] = df['pct_walked_wrkr_cmute_01_to_14_min'] * \
                        df['pct_walked_wrkr_cmute_15_to_29_min'] * \
                        df['pct_walked_wrkr_cmute_30_to_44_min'] * \
                        df['pct_walked_wrkr_cmute_44_to_59_min'] * \
                        df['pct_walked_wrkr_cmute_over_60_min'] * df['zero_check']

# Taxicab Motorcycle Bicycle Or Other Means.19,ACS19_5yr_B08134111
df['ttl_misc_trspt_method_wrkr_cmute'] = df['ACS19_5yr_B08134111']
# Taxicab Motorcycle Bicycle Or Other Means: Less Than 10 Minutes,ACS19_5yr_B08134112
df['ttl_misc_trspt_method_wrkr_cmute_under_10_min'] = df['ACS19_5yr_B08134112']
# Taxicab Motorcycle Bicycle Or Other Means: 10 To 14 Minutes,ACS19_5yr_B08134113
df['ttl_misc_trspt_method_wrkr_cmute_10_to_14_min'] = df['ACS19_5yr_B08134113']
# Taxicab Motorcycle Bicycle Or Other Means: 15 To 19 Minutes,ACS19_5yr_B08134114
df['ttl_misc_trspt_method_wrkr_cmute_15_to_14_min'] = df['ACS19_5yr_B08134114']
# Taxicab Motorcycle Bicycle Or Other Means: 20 To 24 Minutes,ACS19_5yr_B08134115
df['ttl_misc_trspt_method_wrkr_cmute_20_to_24_min'] = df['ACS19_5yr_B08134115']
# Taxicab Motorcycle Bicycle Or Other Means: 25 To 29 Minutes,ACS19_5yr_B08134116
df['ttl_misc_trspt_method_wrkr_cmute_25_to_29_min'] = df['ACS19_5yr_B08134116']
# Taxicab Motorcycle Bicycle Or Other Means: 30 To 34 Minutes,ACS19_5yr_B08134117
df['ttl_misc_trspt_method_wrkr_cmute_30_to_34_min'] = df['ACS19_5yr_B08134117']
# Taxicab Motorcycle Bicycle Or Other Means: 35 To 44 Minutes,ACS19_5yr_B08134118
df['ttl_misc_trspt_method_wrkr_cmute_35_to_44_min'] = df['ACS19_5yr_B08134118']
# Taxicab Motorcycle Bicycle Or Other Means: 45 To 59 Minutes,ACS19_5yr_B08134119
df['ttl_misc_trspt_method_wrkr_cmute_45_to_59_min'] = df['ACS19_5yr_B08134119']
# Taxicab Motorcycle Bicycle Or Other Means: 60 Or More Minutes,ACS19_5yr_B08134120
df['ttl_misc_trspt_method_wrkr_cmute_over_60_min'] = df['ACS19_5yr_B08134120']
# 15 min range Percents
df['pct_misc_trspt_method_wrkr_cmute'] = df['ttl_misc_trspt_method_wrkr_cmute'] / df['ttl_wrkr_cmute']
df['pct_misc_trspt_method_wrkr_cmute_01_to_14_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_under_10_min'] + df['ttl_misc_trspt_method_wrkr_cmute_10_to_14_min']) / df['ttl_wrkr_cmute']
df['pct_misc_trspt_method_wrkr_cmute_15_to_29_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_15_to_14_min'] + df['ttl_misc_trspt_method_wrkr_cmute_20_to_24_min'] + df['ttl_misc_trspt_method_wrkr_cmute_25_to_29_min']) / df['ttl_wrkr_cmute']
df['pct_misc_trspt_method_wrkr_cmute_30_to_44_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_30_to_34_min'] + df['ttl_misc_trspt_method_wrkr_cmute_35_to_44_min']) / df['ttl_wrkr_cmute']
df['pct_misc_trspt_method_wrkr_cmute_44_to_59_min'] = df['ttl_misc_trspt_method_wrkr_cmute_45_to_59_min'] / df['ttl_wrkr_cmute']
df['pct_misc_trspt_method_wrkr_cmute_over_60_min'] = df['ttl_misc_trspt_method_wrkr_cmute_over_60_min'] / df['ttl_wrkr_cmute']
# # 15 min range Percents
# df['pct_misc_trspt_method_wrkr_cmute_01_to_14_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_under_10_min'] + df['ttl_misc_trspt_method_wrkr_cmute_10_to_14_min']) / df['ttl_misc_trspt_method_wrkr_cmute']
# df['pct_misc_trspt_method_wrkr_cmute_15_to_29_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_15_to_14_min'] + df['ttl_misc_trspt_method_wrkr_cmute_20_to_24_min'] + df['ttl_misc_trspt_method_wrkr_cmute_25_to_29_min']) / df['ttl_misc_trspt_method_wrkr_cmute']
# df['pct_misc_trspt_method_wrkr_cmute_30_to_44_min'] = (df['ttl_misc_trspt_method_wrkr_cmute_30_to_34_min'] + df['ttl_misc_trspt_method_wrkr_cmute_35_to_44_min']) / df['ttl_misc_trspt_method_wrkr_cmute']
# df['pct_misc_trspt_method_wrkr_cmute_44_to_59_min'] = df['ttl_misc_trspt_method_wrkr_cmute_45_to_59_min'] / df['ttl_misc_trspt_method_wrkr_cmute']
# df['pct_misc_trspt_method_wrkr_cmute_over_60_min'] = df['ACS19_5yr_B08134120'] / df['ttl_misc_trspt_method_wrkr_cmute']


# df = df[df['zero_check'] !=0]
# df = df.loc[(df[['zero_check']] != 0).all(axis=1)]
# df = df.dropna(subset=['zero_check', 'aggregate_time_commute', 'aggregate_time_commute_drive_alone', 'aggregate_time_commute_carpool', 'aggregate_time_commute_public_transit', 'aggregate_time_commute_other_means'])

##### UNEMPLOYMENT DATA
# # Population 16 Years and Over: in Labor Force - ACS19_5yr_B23025002
# df['population_in_labor_force'] = df['ACS19_5yr_B23025002']
# Population 16 Years and Over: in Labor Force: Civilian Labor Force - ACS19_5yr_B23025003
df['population_in_labor_force'] = df['ACS19_5yr_B23025003']
# Population 16 Years and Over: in Labor Force: Civilian Labor Force: Employed - ACS19_5yr_B23025004
df['population_employed'] = df['ACS19_5yr_B23025004']
# Population 16 Years and Over: in Labor Force: Civilian Labor Force: Unemployed - ACS19_5yr_B23025005
df['population_unemployed'] = df['ACS19_5yr_B23025005']
df['pct_unemployed'] = df['population_unemployed'] / df['population_in_labor_force'] * 100


df['population_density'] = df['total_population'] / df['land_area']# / 1000000 # convert to sqkm
df['worker_density'] = df['total_workers'] / df['land_area']# / 1000000 # convert to sqkm
df['vehicles_density'] = df['aggregate_num_vehicles'] / df['land_area']# / 1000000 # convert to sqkm
df['vehicles_per_worker'] = df['aggregate_num_vehicles'] / df['total_workers']
df['log_vehicles_per_worker'] = np.log(df['vehicles_per_worker'])
df['vehicles_per_worker_x_landarea'] = (df['aggregate_num_vehicles'] / df['total_workers']) * df['land_area'] # / 1000000 # convert to sqkm
df['vehicles_per_worker_per_landarea'] = (df['aggregate_num_vehicles'] / df['total_workers']) / df['land_area'] # / 1000000 # convert to sqkm
df['vehicles_x_worker_per_landarea'] = (df['aggregate_num_vehicles'] * df['total_workers']) / df['land_area'] # / 1000000 # convert to sqkm
df['vehicle_density_x_vehicles_per_worker'] = (df['vehicles_density'] * df['vehicles_per_worker'])

df['average_commute_time_per_worker_drive_alone'] = df['aggregate_time_commute_drive_alone'] / df['total_workers_drove_alone']
average_commute_time_per_worker_drive_alone = sum(df['aggregate_time_commute_drive_alone']) / sum(df['total_workers_drove_alone'])

df['average_commute_time_per_worker_carpool'] = df['aggregate_time_commute_carpool'] / df['total_workers_carpooled']
average_commute_time_per_worker_carpool = sum(df['aggregate_time_commute_carpool']) / sum(df['total_workers_carpooled'])

df['average_commute_time_per_worker_public_transit'] = df['aggregate_time_commute_public_transit'] / df['total_workers_public_transit']
average_commute_time_per_worker_public_transit = sum(df['aggregate_time_commute_public_transit']) / sum(df['total_workers_public_transit'])

df['average_commute_time_per_worker_other_means'] = df['aggregate_time_commute_other_means'] / df['total_workers_other_means']
average_commute_time_per_worker_other_means = sum(df['aggregate_time_commute_other_means']) / sum(df['total_workers_other_means'])


c:\Users\cjong\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\cjong\AppData\Local\Temp\ipykernel_18792\3977577226.py:188: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['log_pct_car_wrkr_cmute_15_to_29_min']  = np.log(df['pct_car_wrkr_cmute_15_to_29_min'])
c:\Users\cjong\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\cjong\AppData\Local\Temp\ipykernel_18792\3977577226.py:189: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

# Data Cleanup involves combining data from two different years

In [ ]:
##### Data Cleanup

df = df.dropna(subset=['vehicles_per_worker',
                                 'total_workers',
                                 'land_area',
                                 'pct_unemployed',
                                 'average_commute_time_per_worker_drive_alone',
                                 'pct_car_wrkr_cmute_01_to_14_min',
                                 'pct_car_wrkr_cmute_15_to_29_min',
                                 'pct_car_wrkr_cmute_30_to_44_min',
                                 'pct_car_wrkr_cmute_44_to_59_min',
                                 'pct_car_wrkr_cmute_over_60_min',
                                 'total_population'])

df['total_workers'] = df['total_workers']
df['land_area'] = df['land_area']
df['vehicles_per_worker'] = df['vehicles_per_worker']
df['average_commute_time_per_worker_drive_alone'] = df['average_commute_time_per_worker_drive_alone']
df['pct_car_wrkr_cmute_01_to_14_min'] = df['pct_car_wrkr_cmute_01_to_14_min']
df['pct_car_wrkr_cmute_15_to_29_min'] = df['pct_car_wrkr_cmute_15_to_29_min']
df['pct_car_wrkr_cmute_44_to_59_min'] = df['pct_car_wrkr_cmute_44_to_59_min']
df['pct_car_wrkr_cmute_30_to_44_min'] = df['pct_car_wrkr_cmute_30_to_44_min']
df['pct_car_wrkr_cmute_over_60_min'] = df['pct_car_wrkr_cmute_over_60_min']
df['pct_unemployed'] = df['pct_unemployed']
df['total_population'] = df['total_population']
df['year'] = 2019

# OLS Regression - Multivariate

In [ ]:
df = df

In [ ]:
lm_ols = smf.ols(formula = 'average_commute_time_per_worker_drive_alone ~ vehicles_per_worker + ', data = df).fit()
print (lm_ols.summary())

                                         OLS Regression Results                                        
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                       0.000
Model:                                                     OLS   Adj. R-squared:                 -0.001
Method:                                          Least Squares   F-statistic:                    0.1745
Date:                                         Mon, 12 Dec 2022   Prob (F-statistic):              0.676
Time:                                                 22:33:12   Log-Likelihood:                -4397.4
No. Observations:                                         1404   AIC:                             8799.
Df Residuals:                                             1402   BIC:                             8809.
Df Model:                                                    1                                         
Covariance Type:                                     nonrobust  

In [ ]:
lm_ols = smf.ols(formula = 'average_commute_time_per_worker_drive_alone ~ vehicles_per_worker + total_population', data = df).fit()
print (lm_ols.summary())

                                         OLS Regression Results                                        
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                       0.032
Model:                                                     OLS   Adj. R-squared:                  0.032
Method:                                          Least Squares   F-statistic:                     49.02
Date:                                         Thu, 08 Dec 2022   Prob (F-statistic):           1.14e-21
Time:                                                 03:03:04   Log-Likelihood:                -9149.4
No. Observations:                                         2949   AIC:                         1.830e+04
Df Residuals:                                             2946   BIC:                         1.832e+04
Df Model:                                                    2                                         
Covariance Type:                                     nonrobust  

In [ ]:
lm_ols = smf.ols(formula = 'average_commute_time_per_worker_drive_alone ~ vehicles_per_worker * total_population', data = df).fit()
print (lm_ols.summary())

                                         OLS Regression Results                                        
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                       0.052
Model:                                                     OLS   Adj. R-squared:                  0.051
Method:                                          Least Squares   F-statistic:                     53.66
Date:                                         Thu, 08 Dec 2022   Prob (F-statistic):           9.21e-34
Time:                                                 03:03:04   Log-Likelihood:                -9119.2
No. Observations:                                         2949   AIC:                         1.825e+04
Df Residuals:                                             2945   BIC:                         1.827e+04
Df Model:                                                    3                                         
Covariance Type:                                     nonrobust  

In [ ]:
df['ln_average_commute_time_per_worker_drive_alone'] = np.log(df['average_commute_time_per_worker_drive_alone']) 
df['ln_vehicles_per_worker'] = np.log(df['vehicles_per_worker']) 

In [ ]:
lm_ols2 = smf.ols(formula = 'ln_average_commute_time_per_worker_drive_alone ~ ln_vehicles_per_worker', data = df).fit()
print (lm_ols2.summary())

                                          OLS Regression Results                                          
Dep. Variable:     ln_average_commute_time_per_worker_drive_alone   R-squared:                       0.004
Model:                                                        OLS   Adj. R-squared:                  0.003
Method:                                             Least Squares   F-statistic:                     10.99
Date:                                            Thu, 08 Dec 2022   Prob (F-statistic):           0.000927
Time:                                                    03:03:05   Log-Likelihood:                 37.953
No. Observations:                                            2949   AIC:                            -71.91
Df Residuals:                                                2947   BIC:                            -59.93
Df Model:                                                       1                                         
Covariance Type:                     

# OLS Regression - Modulating Variables

In [ ]:
# %pip install linearmodels

df.to_csv('file_name.csv')


In [ ]:
import linearmodels as lm

from linearmodels import PanelOLS
from linearmodels import RandomEffects
from linearmodels import FirstDifferenceOLS

In [ ]:
# df_2014['vehicles_per_worker']

In [ ]:
df.head()
df.shape

(2949, 16)

In [ ]:
df = df.set_index(['Geo_FIPS', 'year'])
lm_fd = FirstDifferenceOLS.from_formula('average_commute_time_per_worker_drive_alone ~ vehicles_per_worker', df)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                                  FirstDifferenceOLS Estimation Summary                                  
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                        0.0227
Estimator:                                  FirstDifferenceOLS   R-squared (Between):             -0.8990
No. Observations:                                          781   R-squared (Within):               0.0227
Date:                                         Wed, Dec 07 2022   R-squared (Overall):             -0.8896
Time:                                                 13:22:08   Log-likelihood                   -1649.4
Cov. Estimator:                                      Clustered                                           
                                                                 F-statistic:                      18.145
Entities:                                                 2168   P-value                           0.0000
Avg Obs:                                      

In [ ]:
lm_fd2 = FirstDifferenceOLS.from_formula('ln_average_commute_time_per_worker_drive_alone ~ ln_vehicles_per_worker', df)
res_fd2 = lm_fd2.fit(cov_type='clustered', cluster_entity=True)
print(res_fd2)

                                   FirstDifferenceOLS Estimation Summary                                    
Dep. Variable:     ln_average_commute_time_per_worker_drive_alone   R-squared:                        0.0181
Estimator:                                     FirstDifferenceOLS   R-squared (Between):              0.0347
No. Observations:                                             781   R-squared (Within):               0.0181
Date:                                            Wed, Dec 07 2022   R-squared (Overall):              0.0356
Time:                                                    13:22:08   Log-likelihood                    830.39
Cov. Estimator:                                         Clustered                                           
                                                                    F-statistic:                      14.377
Entities:                                                    2168   P-value                           0.0002
Avg Obs:           

# Completion of homework, further self data analysis

In [ ]:
lm_fd = FirstDifferenceOLS.from_formula('average_commute_time_per_worker_drive_alone ~ vehicles_per_worker + pct_unemployed', df)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                                  FirstDifferenceOLS Estimation Summary                                  
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                        0.1636
Estimator:                                  FirstDifferenceOLS   R-squared (Between):             -1.1397
No. Observations:                                          781   R-squared (Within):               0.1636
Date:                                         Wed, Dec 07 2022   R-squared (Overall):             -1.1271
Time:                                                 13:22:08   Log-likelihood                   -1588.6
Cov. Estimator:                                      Clustered                                           
                                                                 F-statistic:                      76.192
Entities:                                                 2168   P-value                           0.0000
Avg Obs:                                      

In [ ]:
lm_fd = FirstDifferenceOLS.from_formula('average_commute_time_per_worker_drive_alone ~ vehicles_per_worker * pct_unemployed', df)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                                  FirstDifferenceOLS Estimation Summary                                  
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                        0.1674
Estimator:                                  FirstDifferenceOLS   R-squared (Between):             -1.3692
No. Observations:                                          781   R-squared (Within):               0.1674
Date:                                         Wed, Dec 07 2022   R-squared (Overall):             -1.3550
Time:                                                 13:22:08   Log-likelihood                   -1586.9
Cov. Estimator:                                      Clustered                                           
                                                                 F-statistic:                      52.128
Entities:                                                 2168   P-value                           0.0000
Avg Obs:                                      

In [ ]:
lm_fd = FirstDifferenceOLS.from_formula('average_commute_time_per_worker_drive_alone ~ vehicles_per_worker + total_workers + pct_unemployed', df)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                                  FirstDifferenceOLS Estimation Summary                                  
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                        0.1799
Estimator:                                  FirstDifferenceOLS   R-squared (Between):             -0.9826
No. Observations:                                          781   R-squared (Within):               0.1799
Date:                                         Wed, Dec 07 2022   R-squared (Overall):             -0.9577
Time:                                                 13:22:09   Log-likelihood                   -1580.9
Cov. Estimator:                                      Clustered                                           
                                                                 F-statistic:                      56.907
Entities:                                                 2168   P-value                           0.0000
Avg Obs:                                      

In [ ]:
lm_fd = FirstDifferenceOLS.from_formula('average_commute_time_per_worker_drive_alone ~ vehicles_per_worker * land_area', df)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                                  FirstDifferenceOLS Estimation Summary                                  
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                        0.0239
Estimator:                                  FirstDifferenceOLS   R-squared (Between):             -0.8654
No. Observations:                                          781   R-squared (Within):               0.0239
Date:                                         Wed, Dec 07 2022   R-squared (Overall):             -0.8435
Time:                                                 13:22:09   Log-likelihood                   -1649.0
Cov. Estimator:                                      Clustered                                           
                                                                 F-statistic:                      6.3393
Entities:                                                 2168   P-value                           0.0003
Avg Obs:                                      

In [ ]:
lm_fd = FirstDifferenceOLS.from_formula('average_commute_time_per_worker_drive_alone ~ vehicles_per_worker * total_workers', df)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                                  FirstDifferenceOLS Estimation Summary                                  
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                        0.0901
Estimator:                                  FirstDifferenceOLS   R-squared (Between):             -0.6009
No. Observations:                                          781   R-squared (Within):               0.0901
Date:                                         Wed, Dec 07 2022   R-squared (Overall):             -0.5748
Time:                                                 13:22:09   Log-likelihood                   -1621.5
Cov. Estimator:                                      Clustered                                           
                                                                 F-statistic:                      25.686
Entities:                                                 2168   P-value                           0.0000
Avg Obs:                                      

In [ ]:
lm_fd = FirstDifferenceOLS.from_formula('average_commute_time_per_worker_drive_alone ~ vehicles_per_worker * total_workers + vehicles_per_worker * pct_unemployed', df)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                                  FirstDifferenceOLS Estimation Summary                                  
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                        0.1849
Estimator:                                  FirstDifferenceOLS   R-squared (Between):             -0.8769
No. Observations:                                          781   R-squared (Within):               0.1849
Date:                                         Wed, Dec 07 2022   R-squared (Overall):             -0.8529
Time:                                                 13:22:09   Log-likelihood                   -1578.5
Cov. Estimator:                                      Clustered                                           
                                                                 F-statistic:                      35.211
Entities:                                                 2168   P-value                           0.0000
Avg Obs:                                      

In [ ]:
lm_fd = FirstDifferenceOLS.from_formula('average_commute_time_per_worker_drive_alone ~ vehicles_per_worker + pct_unemployed + land_area + total_workers', df)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                                  FirstDifferenceOLS Estimation Summary                                  
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                        0.1813
Estimator:                                  FirstDifferenceOLS   R-squared (Between):             -1.0156
No. Observations:                                          781   R-squared (Within):               0.1813
Date:                                         Wed, Dec 07 2022   R-squared (Overall):             -0.9750
Time:                                                 13:22:10   Log-likelihood                   -1580.3
Cov. Estimator:                                      Clustered                                           
                                                                 F-statistic:                      43.013
Entities:                                                 2168   P-value                           0.0000
Avg Obs:                                      

In [ ]:
lm_fd = FirstDifferenceOLS.from_formula('average_commute_time_per_worker_drive_alone ~ vehicles_per_worker + pct_unemployed + land_area + total_workers', df)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                                  FirstDifferenceOLS Estimation Summary                                  
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                        0.1813
Estimator:                                  FirstDifferenceOLS   R-squared (Between):             -1.0156
No. Observations:                                          781   R-squared (Within):               0.1813
Date:                                         Wed, Dec 07 2022   R-squared (Overall):             -0.9750
Time:                                                 13:22:10   Log-likelihood                   -1580.3
Cov. Estimator:                                      Clustered                                           
                                                                 F-statistic:                      43.013
Entities:                                                 2168   P-value                           0.0000
Avg Obs:                                      

In [ ]:
lm_fd = FirstDifferenceOLS.from_formula('average_commute_time_per_worker_drive_alone ~ vehicles_per_worker * total_workers + pct_unemployed + land_area + vehicles_per_worker * total_workers', df)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                                  FirstDifferenceOLS Estimation Summary                                  
Dep. Variable:     average_commute_time_per_worker_drive_alone   R-squared:                        0.1862
Estimator:                                  FirstDifferenceOLS   R-squared (Between):             -0.9432
No. Observations:                                          781   R-squared (Within):               0.1862
Date:                                         Wed, Dec 07 2022   R-squared (Overall):             -0.9034
Time:                                                 13:22:10   Log-likelihood                   -1577.9
Cov. Estimator:                                      Clustered                                           
                                                                 F-statistic:                      35.515
Entities:                                                 2168   P-value                           0.0000
Avg Obs:                                      